<a href="https://colab.research.google.com/github/shanto12/Baton_Master_Unity/blob/master/tencent_video_unrestricted_opus_10s_trial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# HunyuanVideo - H100 40GB OPTIMIZED
# 5-second NSFW video with face

import subprocess
import sys
import os
import time
import json
import urllib.request
import threading
import glob
from datetime import datetime
from base64 import b64encode
from IPython.display import HTML, display

# ============================================================
# H100 OPTIMIZED SETTINGS - 5 SECOND VIDEO
# ============================================================
VIDEO_PROMPT = """Close-up portrait of a beautiful woman's face, seductive expression,
bedroom eyes looking at camera, soft parted lips, beautiful facial features,
nude shoulders visible, large natural breasts partially visible in frame,
sensual slow movements, soft golden lighting on face,
shallow depth of field, cinematic portrait, photorealistic skin texture,
high detail face, clear facial features, no blur, 8K quality, masterpiece"""

# H100 40GB - can handle more!
WIDTH, HEIGHT = 544, 320      # Good resolution
NUM_FRAMES = 121              # ~5 seconds at 24fps
STEPS = 25                    # Good quality
GUIDANCE_SCALE = 6.0
FLOW_SHIFT = 9.0

# ============================================================
# PATHS
# ============================================================
LOCAL_COMFY = '/content/ComfyUI'
DRIVE_MODELS = '/content/drive/MyDrive/ComfyUI/ComfyUI/models'
OUTPUT_DIR = '/content/drive/MyDrive/AI_Videos_Comfy'

# ============================================================
# LOGGING
# ============================================================
START_TIME = time.time()
SERVER_LOG = []

def log(msg, level="INFO"):
    elapsed = time.time() - START_TIME
    timestamp = datetime.now().strftime("%H:%M:%S")
    print(f"[{timestamp}] [{elapsed:6.1f}s] [{level:5s}] {msg}")

def log_resources():
    try:
        result = subprocess.run(['nvidia-smi', '--query-gpu=memory.used,memory.total,utilization.gpu,name',
                                 '--format=csv,noheader,nounits'], capture_output=True, text=True)
        if result.returncode == 0:
            parts = result.stdout.strip().split(', ')
            gpu_used, gpu_total, gpu_util = int(parts[0]), int(parts[1]), parts[2]
            gpu_name = parts[3] if len(parts) > 3 else "Unknown"
            gpu_pct = 100 * gpu_used / gpu_total
            log(f"📊 GPU: {gpu_used:,}/{gpu_total:,} MB ({gpu_pct:.0f}%) | {gpu_name}", "STATS")
    except:
        pass

print("="*70)
print("🚀 HUNYUANVIDEO - H100 40GB OPTIMIZED")
print("="*70)
log(f"Settings: {WIDTH}x{HEIGHT}, {NUM_FRAMES} frames ({NUM_FRAMES/24:.1f}s), {STEPS} steps")
log_resources()
print("="*70)

# ============================================================
# SETUP
# ============================================================
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "gitpython", "requests", "websocket-client"], capture_output=True)

import torch
import shutil

torch.cuda.empty_cache()
log("✅ Dependencies ready")

# Mount Drive
from google.colab import drive, runtime
if not os.path.exists('/content/drive/MyDrive'):
    drive.mount('/content/drive')
os.makedirs(OUTPUT_DIR, exist_ok=True)
log("✅ Drive mounted")

# HF Auth
try:
    from google.colab import userdata
    from huggingface_hub import login
    login(token=userdata.get('HF_TOKEN'), add_to_git_credential=False)
except:
    pass

# Kill existing
subprocess.run(["pkill", "-9", "-f", "main.py"], capture_output=True)
time.sleep(2)

# Setup ComfyUI locally (fast)
if os.path.exists(LOCAL_COMFY):
    shutil.rmtree(LOCAL_COMFY)

log("Setting up ComfyUI locally...")
subprocess.run(["git", "clone", "--depth", "1", "https://github.com/comfyanonymous/ComfyUI", LOCAL_COMFY],
               capture_output=True, timeout=120)
os.chdir(LOCAL_COMFY)
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "-r", "requirements.txt"], capture_output=True, timeout=180)

# Custom nodes
os.makedirs('custom_nodes', exist_ok=True)
for repo, name in [
    ("https://github.com/kijai/ComfyUI-HunyuanVideoWrapper", "ComfyUI-HunyuanVideoWrapper"),
    ("https://github.com/Kosinkadink/ComfyUI-VideoHelperSuite", "ComfyUI-VideoHelperSuite"),
]:
    subprocess.run(["git", "clone", "--depth", "1", repo, f"custom_nodes/{name}"], capture_output=True, timeout=60)

for req in ['custom_nodes/ComfyUI-HunyuanVideoWrapper/requirements.txt',
            'custom_nodes/ComfyUI-VideoHelperSuite/requirements.txt']:
    if os.path.exists(req):
        subprocess.run([sys.executable, "-m", "pip", "install", "-q", "-r", req], capture_output=True)

log("✅ ComfyUI ready")

# Link models from Drive
for d in ['vae', 'diffusion_models', 'LLM', 'clip']:
    os.makedirs(f'models/{d}', exist_ok=True)

model_links = [
    (f"{DRIVE_MODELS}/vae/hunyuan_video_vae_bf16.safetensors", "models/vae/hunyuan_video_vae_bf16.safetensors"),
    (f"{DRIVE_MODELS}/diffusion_models/hunyuan_video_720_cfgdistill_fp8_e4m3fn.safetensors", "models/diffusion_models/hunyuan_video_720_cfgdistill_fp8_e4m3fn.safetensors"),
    (f"{DRIVE_MODELS}/LLM/llava-llama-3-8b-text-encoder-tokenizer", "models/LLM/llava-llama-3-8b-text-encoder-tokenizer"),
    (f"{DRIVE_MODELS}/clip/clip-vit-large-patch14", "models/clip/clip-vit-large-patch14"),
]

for src, dst in model_links:
    if os.path.exists(src):
        if os.path.exists(dst):
            os.remove(dst) if os.path.isfile(dst) else shutil.rmtree(dst)
        os.symlink(src, dst)
        log(f"  ✓ Linked {os.path.basename(src)}")
    else:
        log(f"  ✗ Missing: {os.path.basename(src)}", "WARN")

log("✅ Models linked")
log_resources()

# ============================================================
# START SERVER
# ============================================================
log("Starting ComfyUI server...")

import socket
port = 8188
subprocess.run(["fuser", "-k", f"{port}/tcp"], capture_output=True)
time.sleep(1)

server_process = subprocess.Popen(
    ["python", "main.py", "--listen", "127.0.0.1", "--port", str(port),
     "--output-directory", OUTPUT_DIR, "--disable-auto-launch"],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1
)

def read_server_output():
    for line in server_process.stdout:
        line = line.strip()
        if line:
            SERVER_LOG.append(line)

output_thread = threading.Thread(target=read_server_output, daemon=True)
output_thread.start()

server_address = f"127.0.0.1:{port}"

server_ready = False
for i in range(180):
    if server_process.poll() is not None:
        log(f"❌ Server died!", "ERROR")
        for line in SERVER_LOG[-15:]:
            log(f"   {line[:100]}", "ERROR")
        break
    try:
        urllib.request.urlopen(f"http://{server_address}/system_stats", timeout=2)
        log(f"✅ Server ready in {i}s!")
        server_ready = True
        break
    except:
        if i % 20 == 0 and i > 0:
            log(f"  Loading... ({i}s)")
        time.sleep(1)

if not server_ready:
    raise Exception("Server failed")

# ============================================================
# DISCOVER NODES
# ============================================================
time.sleep(2)
log("Discovering nodes...")

with urllib.request.urlopen(f"http://{server_address}/object_info", timeout=60) as r:
    available_nodes = json.loads(r.read())

scheduler_options = []
if 'HyVideoSampler' in available_nodes:
    sampler_info = available_nodes['HyVideoSampler']
    if 'input' in sampler_info and 'required' in sampler_info['input']:
        for param, info in sampler_info['input']['required'].items():
            if param == 'scheduler' and isinstance(info, list) and len(info) > 0:
                scheduler_options = info[0] if isinstance(info[0], list) else []

SCHEDULER = scheduler_options[0] if scheduler_options else "FlowMatchDiscreteScheduler"
log(f"  Scheduler: {SCHEDULER}")
log(f"✅ Found {len([k for k in available_nodes if 'hyvideo' in k.lower()])} HunyuanVideo nodes")

# ============================================================
# BUILD WORKFLOW - H100 OPTIMIZED
# ============================================================
log("Building H100-optimized workflow...")

import random
SEED = random.randint(0, 2**63-1)
clean_prompt = " ".join(VIDEO_PROMPT.split())

workflow = {
    "1": {
        "class_type": "HyVideoModelLoader",
        "inputs": {
            "model": "hunyuan_video_720_cfgdistill_fp8_e4m3fn.safetensors",
            "base_precision": "bf16",          # H100 handles bf16 great
            "quantization": "disabled",
            "load_device": "main_device",      # Keep on GPU
            "attention_mode": "sdpa",          # Fast attention
        }
    },
    "2": {
        "class_type": "HyVideoVAELoader",
        "inputs": {
            "model_name": "hunyuan_video_vae_bf16.safetensors",
            "precision": "bf16",
        }
    },
    "10": {
        "class_type": "DownloadAndLoadHyVideoTextEncoder",
        "inputs": {
            "llm_model": "Kijai/llava-llama-3-8b-text-encoder-tokenizer",
            "clip_model": "openai/clip-vit-large-patch14",
            "precision": "bf16",
        }
    },
    "3": {
        "class_type": "HyVideoTextEncode",
        "inputs": {
            "text_encoders": ["10", 0],
            "prompt": clean_prompt,
            "force_offload": True,             # Free VRAM after encoding
        }
    },
    "4": {
        "class_type": "HyVideoSampler",
        "inputs": {
            "model": ["1", 0],
            "hyvid_embeds": ["3", 0],
            "width": WIDTH,
            "height": HEIGHT,
            "num_frames": NUM_FRAMES,
            "steps": STEPS,
            "embedded_guidance_scale": GUIDANCE_SCALE,
            "flow_shift": FLOW_SHIFT,
            "seed": SEED,
            "scheduler": SCHEDULER,
            "force_offload": True,
            "denoise_strength": 1.0,
        }
    },
    "7": {
        "class_type": "HyVideoDecode",
        "inputs": {
            "vae": ["2", 0],
            "samples": ["4", 0],
            "enable_vae_tiling": True,
            "auto_tile_size": False,
            "spatial_tile_sample_min_size": 128,  # Larger tiles for H100
            "temporal_tiling_sample_size": 32,    # Larger temporal tiles
        }
    },
    "8": {
        "class_type": "VHS_VideoCombine",
        "inputs": {
            "images": ["7", 0],
            "frame_rate": 24,
            "loop_count": 0,
            "filename_prefix": "HunyuanVideo_H100_5sec",
            "format": "video/h264-mp4",
            "save_output": True,
            "pingpong": False,
            "pix_fmt": "yuv420p",
        }
    }
}

log(f"✅ Workflow ready (seed: {SEED})")

# ============================================================
# GENERATE
# ============================================================
print("\n" + "="*70)
log("🚀 STARTING 5-SECOND VIDEO GENERATION (H100)")
print("="*70)
log(f"Prompt: {clean_prompt[:80]}...")
log(f"Resolution: {WIDTH}x{HEIGHT}, {NUM_FRAMES} frames ({NUM_FRAMES/24:.1f}s), {STEPS} steps")
log(f"⚡ H100 should complete in ~10-20 minutes")
log_resources()

existing_videos = set(glob.glob(f"{OUTPUT_DIR}/**/*.mp4", recursive=True))

log("Queueing prompt...")
try:
    data = json.dumps({"prompt": workflow}).encode()
    req = urllib.request.Request(f"http://{server_address}/prompt", data=data)
    req.add_header('Content-Type', 'application/json')
    response = urllib.request.urlopen(req, timeout=30)
    result = json.loads(response.read())
    prompt_id = result['prompt_id']
    log(f"✅ Queued: {prompt_id}")
except urllib.error.HTTPError as e:
    err = json.loads(e.read().decode())
    log(f"❌ Queue failed: {err}", "ERROR")
    raise

log("⏳ Generating 5-second video...")
gen_start = time.time()
last_log = 0

def get_history(pid):
    try:
        with urllib.request.urlopen(f"http://{server_address}/history/{pid}", timeout=10) as r:
            return json.loads(r.read())
    except:
        return {}

def get_queue():
    try:
        with urllib.request.urlopen(f"http://{server_address}/queue", timeout=10) as r:
            return json.loads(r.read())
    except:
        return {}

while True:
    time.sleep(5)
    elapsed = time.time() - gen_start

    if elapsed - last_log >= 30:
        log(f"⏳ {elapsed/60:.1f} min elapsed")
        log_resources()

        progress = [l for l in SERVER_LOG[-20:] if any(x in l.lower() for x in
            ['sampling', '%', 'step', 'it/s', 'loading', 'decod', 'vae', '|'])]
        for line in progress[-2:]:
            log(f"   [SERVER] {line[:80]}", "DEBUG")
        last_log = elapsed

    history = get_history(prompt_id)
    if prompt_id in history:
        if history[prompt_id].get('outputs'):
            log("✅ Generation complete!")
            break
        status = history[prompt_id].get('status', {})
        if status.get('status_str') == 'error':
            log(f"❌ Error!", "ERROR")
            for line in SERVER_LOG[-30:]:
                if 'error' in line.lower() or 'oom' in line.lower():
                    log(f"   {line[:100]}", "ERROR")
            break

    queue = get_queue()
    if elapsed > 300 and not queue.get('queue_running') and not queue.get('queue_pending'):
        if get_history(prompt_id).get(prompt_id, {}).get('outputs'):
            break
        errors = [l for l in SERVER_LOG[-30:] if 'error' in l.lower() or 'oom' in l.lower()]
        if errors:
            log("❌ Failed", "ERROR")
            for e in errors[-5:]:
                log(f"   {e[:100]}", "ERROR")
            break

    if elapsed > 3600:  # 1 hour timeout
        log("❌ Timeout!", "ERROR")
        break

gen_time = time.time() - gen_start

# ============================================================
# FIND AND PLAY VIDEO
# ============================================================
log("Looking for video...")

video_path = None

history = get_history(prompt_id)
if prompt_id in history:
    for node_output in history[prompt_id].get('outputs', {}).values():
        for key in ['gifs', 'videos', 'images']:
            if key in node_output:
                for item in node_output[key]:
                    fn = item.get('filename', '')
                    if os.path.exists(os.path.join(OUTPUT_DIR, fn)):
                        video_path = os.path.join(OUTPUT_DIR, fn)
                        break

if not video_path:
    for f in glob.glob(os.path.join(OUTPUT_DIR, '**/*.mp4'), recursive=True):
        if f not in existing_videos:
            video_path = f
            break

if video_path and os.path.exists(video_path):
    size = os.path.getsize(video_path) / 1e6
    log(f"✅ Video: {video_path} ({size:.1f} MB)")

    try:
        with open(video_path, "rb") as f:
            video_b64 = b64encode(f.read()).decode()

        display(HTML(f'''
        <div style="background:#1a1a2e;padding:20px;border-radius:10px;margin:10px 0;">
            <h2 style="color:#00ff88;">🚀 5-SECOND NSFW VIDEO (H100)</h2>
            <video width="720" controls autoplay loop style="border-radius:8px;border:2px solid #00ff88;">
                <source src="data:video/mp4;base64,{video_b64}" type="video/mp4">
            </video>
            <p style="color:#aaa;margin-top:10px;">
                📁 {video_path}<br>
                📊 {size:.1f} MB | ⏱️ {gen_time/60:.1f} min | 🎬 {NUM_FRAMES} frames ({NUM_FRAMES/24:.1f}s)<br>
                🖥️ H100 40GB | {WIDTH}x{HEIGHT} | {STEPS} steps
            </p>
        </div>
        '''))
        log("✅ Video displayed!")
    except Exception as e:
        log(f"Display error: {e}", "WARN")
else:
    log("❌ No video found!", "ERROR")
    for line in SERVER_LOG[-30:]:
        log(f"   {line[:100]}", "DEBUG")

# ============================================================
# DONE
# ============================================================
print("\n" + "="*70)
log("🚀 DONE!" if video_path else "⚠️ ISSUES")
print("="*70)
log(f"Total: {(time.time()-START_TIME)/60:.1f} min | Gen: {gen_time/60:.1f} min")

if video_path:
    log(f"📁 Saved to: {video_path}")
    log("🔌 Disconnecting in 30s to save compute units...")
    time.sleep(30)
    server_process.terminate()
    runtime.unassign()
else:
    log("⚠️ Not disconnecting - check errors above")

🚀 HUNYUANVIDEO - H100 40GB OPTIMIZED
[13:58:28] [   0.0s] [INFO ] Settings: 544x320, 121 frames (5.0s), 25 steps
[13:58:28] [   0.0s] [STATS] 📊 GPU: 0/40,960 MB (0%) | NVIDIA A100-SXM4-40GB
[13:58:34] [   6.5s] [INFO ] ✅ Dependencies ready
Mounted at /content/drive
[13:59:24] [  56.0s] [INFO ] ✅ Drive mounted
[13:59:27] [  59.0s] [INFO ] Setting up ComfyUI locally...
[13:59:46] [  78.0s] [INFO ] ✅ ComfyUI ready
[13:59:48] [  79.9s] [INFO ]   ✓ Linked hunyuan_video_vae_bf16.safetensors
[13:59:48] [  80.2s] [INFO ]   ✓ Linked hunyuan_video_720_cfgdistill_fp8_e4m3fn.safetensors
[13:59:48] [  80.2s] [INFO ]   ✓ Linked llava-llama-3-8b-text-encoder-tokenizer
[13:59:49] [  81.2s] [INFO ]   ✓ Linked clip-vit-large-patch14
[13:59:49] [  81.2s] [INFO ] ✅ Models linked
[13:59:49] [  81.2s] [STATS] 📊 GPU: 0/40,960 MB (0%) | NVIDIA A100-SXM4-40GB
[13:59:49] [  81.2s] [INFO ] Starting ComfyUI server...
[14:00:10] [ 102.2s] [INFO ]   Loading... (20s)
[14:00:18] [ 110.2s] [INFO ] ✅ Server ready in 28

[14:12:22] [ 833.7s] [INFO ] ✅ Video displayed!

[14:12:22] [ 833.7s] [INFO ] 🚀 DONE!
[14:12:22] [ 833.7s] [INFO ] Total: 13.9 min | Gen: 12.0 min
[14:12:22] [ 833.7s] [INFO ] 📁 Saved to: /content/drive/MyDrive/AI_Videos_Comfy/HunyuanVideo_H100_5sec_00001.mp4
[14:12:22] [ 833.7s] [INFO ] 🔌 Disconnecting in 30s to save compute units...
